In [158]:
import pandas as pd
import os
import numpy as np
import re
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder, TargetEncoder 
import matplotlib 

### reading dataset

In [159]:
filename = 'Leads.csv'
folder_name = 'kaggle_dataset'
dataset_loc = os.path.join(folder_name,filename)
dataset = pd.read_csv(dataset_loc)
dataset.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


In [160]:
dataset.shape

(9240, 37)

### check data type of cols and converting it to correct format

In [161]:
dataset.dtypes

Prospect ID                                       object
Lead Number                                        int64
Lead Origin                                       object
Lead Source                                       object
Do Not Email                                      object
Do Not Call                                       object
Converted                                          int64
TotalVisits                                      float64
Total Time Spent on Website                        int64
Page Views Per Visit                             float64
Last Activity                                     object
Country                                           object
Specialization                                    object
How did you hear about X Education                object
What is your current occupation                   object
What matters most to you in choosing a course     object
Search                                            object
Magazine                       

In [162]:
dataset.convert_dtypes().dtypes

Prospect ID                                      string[python]
Lead Number                                               Int64
Lead Origin                                      string[python]
Lead Source                                      string[python]
Do Not Email                                     string[python]
Do Not Call                                      string[python]
Converted                                                 Int64
TotalVisits                                               Int64
Total Time Spent on Website                               Int64
Page Views Per Visit                                    Float64
Last Activity                                    string[python]
Country                                          string[python]
Specialization                                   string[python]
How did you hear about X Education               string[python]
What is your current occupation                  string[python]
What matters most to you in choosing a c

### check if null in target variable -> converted

In [163]:
dataset['Converted'].isnull().sum()

0

### check if null in other columns

In [164]:
dataset.isnull().sum()

Prospect ID                                         0
Lead Number                                         0
Lead Origin                                         0
Lead Source                                        36
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                       137
Total Time Spent on Website                         0
Page Views Per Visit                              137
Last Activity                                     103
Country                                          2461
Specialization                                   1438
How did you hear about X Education               2207
What is your current occupation                  2690
What matters most to you in choosing a course    2709
Search                                              0
Magazine                                            0
Newspaper Article           

### replacing spaces in column names with _ for consistency

In [165]:
## rename cols with _ 
dataset.columns = dataset.columns.str.replace(' ', '_')
dataset.columns

Index(['Prospect_ID', 'Lead_Number', 'Lead_Origin', 'Lead_Source',
       'Do_Not_Email', 'Do_Not_Call', 'Converted', 'TotalVisits',
       'Total_Time_Spent_on_Website', 'Page_Views_Per_Visit', 'Last_Activity',
       'Country', 'Specialization', 'How_did_you_hear_about_X_Education',
       'What_is_your_current_occupation',
       'What_matters_most_to_you_in_choosing_a_course', 'Search', 'Magazine',
       'Newspaper_Article', 'X_Education_Forums', 'Newspaper',
       'Digital_Advertisement', 'Through_Recommendations',
       'Receive_More_Updates_About_Our_Courses', 'Tags', 'Lead_Quality',
       'Update_me_on_Supply_Chain_Content', 'Get_updates_on_DM_Content',
       'Lead_Profile', 'City', 'Asymmetrique_Activity_Index',
       'Asymmetrique_Profile_Index', 'Asymmetrique_Activity_Score',
       'Asymmetrique_Profile_Score',
       'I_agree_to_pay_the_amount_through_cheque',
       'A_free_copy_of_Mastering_The_Interview', 'Last_Notable_Activity'],
      dtype='object')

### seperate numerical vs categorical variables


In [166]:
numerical_cols_df = dataset.select_dtypes(include=[np.number])
categorical_cols_df = dataset.select_dtypes(exclude=[np.number])
numerical_cols_df,categorical_cols_df

(      Lead_Number  Converted  TotalVisits  Total_Time_Spent_on_Website  \
 0          660737          0          0.0                            0   
 1          660728          0          5.0                          674   
 2          660727          1          2.0                         1532   
 3          660719          0          1.0                          305   
 4          660681          1          2.0                         1428   
 ...           ...        ...          ...                          ...   
 9235       579564          1          8.0                         1845   
 9236       579546          0          2.0                          238   
 9237       579545          0          2.0                          199   
 9238       579538          1          3.0                          499   
 9239       579533          1          6.0                         1279   
 
       Page_Views_Per_Visit  Asymmetrique_Activity_Score  \
 0                     0.00           

### substituting categorical variables nan with the most frequent value


In [167]:
most_frequent_si = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
transformed_cols = {}

for num,cat_col_name in enumerate(categorical_cols_df.columns):
    #print(num,cat_col_name)
    nan_removed_col = most_frequent_si.fit_transform(np.array(categorical_cols_df.iloc[:,num]).reshape(-1,1))  
    column_name_key = cat_col_name
    transformed_cols[column_name_key]=nan_removed_col.flatten()

#print(type(transformed_cols.keys)
df_cat = pd.DataFrame.from_dict(transformed_cols)
df_cat.head()

,Prospect_ID,Lead_Origin,Lead_Source,Do_Not_Email,Do_Not_Call,Last_Activity,Country,Specialization,How_did_you_hear_about_X_Education,What_is_your_current_occupation,...,Lead_Quality,Update_me_on_Supply_Chain_Content,Get_updates_on_DM_Content,Lead_Profile,City,Asymmetrique_Activity_Index,Asymmetrique_Profile_Index,I_agree_to_pay_the_amount_through_cheque,A_free_copy_of_Mastering_The_Interview,Last_Notable_Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,API,Olark Chat,No,No,Page Visited on Website,India,Select,Select,Unemployed,...,Low in Relevance,No,No,Select,Select,02.Medium,02.Medium,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,API,Organic Search,No,No,Email Opened,India,Select,Select,Unemployed,...,Might be,No,No,Select,Select,02.Medium,02.Medium,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,Landing Page Submission,Direct Traffic,No,No,Email Opened,India,Business Administration,Select,Student,...,Might be,No,No,Potential Lead,Mumbai,02.Medium,01.High,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,Landing Page Submission,Direct Traffic,No,No,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,...,Not Sure,No,No,Select,Mumbai,02.Medium,01.High,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,Landing Page Submission,Google,No,No,Converted to Lead,India,Select,Other,Unemployed,...,Might be,No,No,Select,Mumbai,02.Medium,01.High,No,No,Modified


In [168]:
df_cat.isnull().sum()

Prospect_ID                                      0
Lead_Origin                                      0
Lead_Source                                      0
Do_Not_Email                                     0
Do_Not_Call                                      0
Last_Activity                                    0
Country                                          0
Specialization                                   0
How_did_you_hear_about_X_Education               0
What_is_your_current_occupation                  0
What_matters_most_to_you_in_choosing_a_course    0
Search                                           0
Magazine                                         0
Newspaper_Article                                0
X_Education_Forums                               0
Newspaper                                        0
Digital_Advertisement                            0
Through_Recommendations                          0
Receive_More_Updates_About_Our_Courses           0
Tags                           

### processing of null in numerical cols
-. null present in all except Lead_Number', 'Converted', Total_Time_Spent_on_Website

In [169]:
numerical_cols_df.columns

Index(['Lead_Number', 'Converted', 'TotalVisits',
       'Total_Time_Spent_on_Website', 'Page_Views_Per_Visit',
       'Asymmetrique_Activity_Score', 'Asymmetrique_Profile_Score'],
      dtype='object')

### substituting null values in TotalVisits, Page views with 0


In [170]:
fill_zero_si = SimpleImputer(missing_values=np.nan,fill_value=0)

total_visits = fill_zero_si.fit_transform(np.array(numerical_cols_df['TotalVisits']).reshape(-1,1)).flatten()
numerical_cols_df['TotalVisits'] = total_visits
numerical_cols_df['TotalVisits'].isnull().sum()

page_views = fill_zero_si.fit_transform(np.array(numerical_cols_df['Page_Views_Per_Visit']).reshape(-1,1)).flatten()
numerical_cols_df['Page_Views_Per_Visit'] = page_views
numerical_cols_df['Page_Views_Per_Visit'].isnull().sum()

0

### substituting null values in Asymmetrique_Activity_Score, Asymmetrique_Profile_Score with median

In [171]:

numerical_cols_df['Median_Asymm_Activity_Score'] = numerical_cols_df['Asymmetrique_Activity_Score'].fillna(value=numerical_cols_df['Asymmetrique_Activity_Score'].median())

In [172]:
numerical_cols_df.drop(labels='Asymmetrique_Activity_Score',axis=1,inplace=True)

In [173]:
numerical_cols_df['Median_Asymm_Profile_Score'] = numerical_cols_df['Asymmetrique_Profile_Score'].fillna(value=numerical_cols_df['Asymmetrique_Profile_Score'].median())
numerical_cols_df.drop(labels='Asymmetrique_Profile_Score',axis=1,inplace=True)

In [174]:
numerical_cols_df.head()

,Lead_Number,Converted,TotalVisits,Total_Time_Spent_on_Website,Page_Views_Per_Visit,Median_Asymm_Activity_Score,Median_Asymm_Profile_Score
0,660737,0,0.0,0,0.0,15.0,15.0
1,660728,0,5.0,674,2.5,15.0,15.0
2,660727,1,2.0,1532,2.0,14.0,20.0
3,660719,0,1.0,305,1.0,13.0,17.0
4,660681,1,2.0,1428,1.0,15.0,18.0


### converting categorical columns to numerical cols

In [175]:
df_cat.columns

Index(['Prospect_ID', 'Lead_Origin', 'Lead_Source', 'Do_Not_Email',
       'Do_Not_Call', 'Last_Activity', 'Country', 'Specialization',
       'How_did_you_hear_about_X_Education', 'What_is_your_current_occupation',
       'What_matters_most_to_you_in_choosing_a_course', 'Search', 'Magazine',
       'Newspaper_Article', 'X_Education_Forums', 'Newspaper',
       'Digital_Advertisement', 'Through_Recommendations',
       'Receive_More_Updates_About_Our_Courses', 'Tags', 'Lead_Quality',
       'Update_me_on_Supply_Chain_Content', 'Get_updates_on_DM_Content',
       'Lead_Profile', 'City', 'Asymmetrique_Activity_Index',
       'Asymmetrique_Profile_Index',
       'I_agree_to_pay_the_amount_through_cheque',
       'A_free_copy_of_Mastering_The_Interview', 'Last_Notable_Activity'],
      dtype='object')

In [176]:
df_cat.shape

(9240, 30)

In [177]:
## *target encoder / proportion of occurance to avoid target leakage: ['Lead_Origin','Lead_Source','Last_Activity','Country','Specialization','How_did_you_hear_about_X_Education','What_is_your_current_occupation','Lead_Profile','City','Last_Notable_Activity','Tags']
## *binary encoder: yes, no: Do_Not_Email, Do_Not_Call, Update_me_on_Supply_Chain_Content, Get_updates_on_DM_Content, I_agree_to_pay_the_amount_through_cheque, A_free_copy_of_Mastering_The_Interview
## *drop cols: Asymmetrique_Profile_Index, Asymmetrique_Activity_Index
## *ordinal encoder: Lead_Quality
## one_hot_encoding: what matters most to you in choosing a career
## *combine cols: ['Search','Magazine','Newspaper','Digital_Advertisement','Newspaper_Article','X_Education_Forums','Through_Recommendations']
# substituting yes with probability (#yes/#obs) for each column..then add across rows to collapse multiple columns into single
# column


In [178]:
df_cat['Do_Not_Email'].replace(to_replace={'No':0,'Yes':1},inplace=True)
df_cat['Do_Not_Call'].replace(to_replace={'No':0,'Yes':1},inplace=True)
df_cat['Update_me_on_Supply_Chain_Content'].replace(to_replace={'No':0,'Yes':1},inplace=True)
df_cat['Get_updates_on_DM_Content'].replace(to_replace={'No':0,'Yes':1},inplace=True)
df_cat['I_agree_to_pay_the_amount_through_cheque'].replace(to_replace={'No':0,'Yes':1},inplace=True)
df_cat['A_free_copy_of_Mastering_The_Interview'].replace(to_replace={'No':0,'Yes':1},inplace=True)
df_cat['Receive_More_Updates_About_Our_Courses'].replace(to_replace={'No':0,'Yes':1},inplace=True)
df_cat[['Do_Not_Email','Do_Not_Call','Update_me_on_Supply_Chain_Content','Get_updates_on_DM_Content','Receive_More_Updates_About_Our_Courses']]


,Do_Not_Email,Do_Not_Call,Update_me_on_Supply_Chain_Content,Get_updates_on_DM_Content,Receive_More_Updates_About_Our_Courses
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
9235,1,0,0,0,0
9236,0,0,0,0,0
9237,1,0,0,0,0
9238,0,0,0,0,0


In [179]:
# LEad quality: 'not sure' and 'maybe' treated as same category
lead_quality_encoding = {'Might be':0,'Not Sure':0,'Worst':1,'Low in Relevance':2,'High in Relevance':3}
df_cat['numeric_lead_quality'] = df_cat['Lead_Quality'].map(lead_quality_encoding)
df_cat[['Lead_Quality','numeric_lead_quality']].head()

,Lead_Quality,numeric_lead_quality
0,Low in Relevance,2
1,Might be,0
2,Might be,0
3,Not Sure,0
4,Might be,0


In [180]:
### Lead source had duplicate values 'google' and 'Google'
df_cat['Lead_Source'].replace(to_replace='google',value='Google',inplace=True)


In [181]:
## converting to numeric based on probability of occurance of the class in the column
columns_to_convert= ['Lead_Origin','Lead_Source','Last_Activity','Country','Specialization','How_did_you_hear_about_X_Education','What_is_your_current_occupation','Lead_Profile','City','Last_Notable_Activity','Tags','What_matters_most_to_you_in_choosing_a_course']
for col in columns_to_convert:
    col_encoding_num=dict((df_cat[col].value_counts()/df_cat.shape[0]))
    #print(col_encoding_num)
    df_cat[col].replace(to_replace=col_encoding_num.keys(),value=col_encoding_num.values(),inplace=True)

In [182]:
cols_encoding_prob = ['Search','Magazine','Newspaper','Digital_Advertisement','Newspaper_Article','X_Education_Forums','Through_Recommendations']
for col in cols_encoding_prob:
    print(df_cat[df_cat[col]=='Yes'].count()[1], df_cat[df_cat[col]=='No'].count()[1])
    df_cat[col].replace(to_replace={'Yes':df_cat[df_cat[col]=='Yes'].count()[0]/df_cat.shape[0]
                                     ,'No':0},inplace=True)
agg_cols = ['Search','Magazine','Newspaper','Digital_Advertisement','Newspaper_Article','X_Education_Forums','Through_Recommendations']
df_cat['ad_medium']=df_cat.loc[:,agg_cols].sum(axis=1)
df_cat[df_cat.ad_medium != 0]

C:\Users\tanm.pha\AppData\Local\Temp\ipykernel_18140\701492066.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df_cat[df_cat[col]=='Yes'].count()[1], df_cat[df_cat[col]=='No'].count()[1])
C:\Users\tanm.pha\AppData\Local\Temp\ipykernel_18140\701492066.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df_cat[df_cat[col]=='Yes'].count()[1], df_cat[df_cat[col]=='No'].count()[1])
C:\Users\tanm.pha\AppData\Local\Temp\ipykernel_18140\701492066.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with

14 9226
0 9240
1 9239
4 9236
2 9238
1 9239
7 9233


C:\Users\tanm.pha\AppData\Local\Temp\ipykernel_18140\701492066.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df_cat[df_cat[col]=='Yes'].count()[1], df_cat[df_cat[col]=='No'].count()[1])
C:\Users\tanm.pha\AppData\Local\Temp\ipykernel_18140\701492066.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_cat[col].replace(to_replace={'Yes':df_cat[df_cat[col]=='Yes'].count()[0]/df_cat.shape[0]
C:\Users\tanm.pha\AppData\Local\Temp\ipykernel_18140\701492066.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consis

,Prospect_ID,Lead_Origin,Lead_Source,Do_Not_Email,Do_Not_Call,Last_Activity,Country,Specialization,How_did_you_hear_about_X_Education,What_is_your_current_occupation,...,Get_updates_on_DM_Content,Lead_Profile,City,Asymmetrique_Activity_Index,Asymmetrique_Profile_Index,I_agree_to_pay_the_amount_through_cheque,A_free_copy_of_Mastering_The_Interview,Last_Notable_Activity,numeric_lead_quality,ad_medium
788,a50571c2-a35f-42e4-9cfb-0308c0c05e4a,0.528788,0.314827,0,0,0.383117,0.968939,0.091775,0.037662,0.897186,...,0,0.741883,0.502381,02.Medium,01.High,0,0,0.305952,0,0.000758
2307,23f45ed6-f34c-4015-9b0a-581cf7fb79ee,0.387446,0.189935,0,0,0.383117,0.968939,0.090693,0.784632,0.076407,...,0,0.174567,0.502381,02.Medium,01.High,0,0,0.305952,2,0.001515
2511,8712596a-6013-4840-a1a5-1ded98a19adf,0.528788,0.275216,0,0,0.297078,0.968939,0.365801,0.784632,0.897186,...,0,0.741883,0.502381,02.Medium,01.High,0,0,0.368723,0,0.001515
2821,93ca59c3-d429-417f-ab3f-46f097a979bb,0.528788,0.275216,0,0,0.383117,0.968939,0.105628,0.784632,0.897186,...,0,0.741883,0.041126,02.Medium,02.Medium,0,1,0.305952,0,0.002273
2897,d0905fc2-e232-45fb-acac-ad1d483e1525,0.528788,0.124892,0,0,0.383117,0.968939,0.105628,0.784632,0.897186,...,0,0.174567,0.502381,02.Medium,02.Medium,0,1,0.305952,3,0.001515
3203,f693004e-815a-4b31-bd50-e9de0f097a6d,0.528788,0.275216,0,0,0.383117,0.968939,0.012121,0.784632,0.897186,...,0,0.174567,0.502381,02.Medium,02.Medium,0,1,0.305952,0,0.001515
3414,d8350bea-ec66-4c36-a2d0-630f8c0e0c93,0.528788,0.275216,0,0,0.383117,0.968939,0.105628,0.784632,0.897186,...,0,0.741883,0.243398,02.Medium,02.Medium,0,0,0.305952,0,0.000108
3829,4ee1b650-b7e5-4a46-9974-d7c2e474b12c,0.528788,0.275216,0,0,0.383117,0.968939,0.105628,0.784632,0.076407,...,0,0.741883,0.502381,02.Medium,02.Medium,0,1,0.305952,0,0.000758
3844,8ea55d70-11b4-48fa-b906-b7ae172b21be,0.528788,0.275216,0,0,0.383117,0.968939,0.043615,0.784632,0.897186,...,0,0.741883,0.243398,02.Medium,02.Medium,0,0,0.305952,0,0.001515
5193,6a8950c8-bc33-4bd0-a998-5ec260f4da03,0.528788,0.275216,0,0,0.069264,0.968939,0.017208,0.087446,0.897186,...,0,0.741883,0.041126,02.Medium,02.Medium,0,1,0.368723,0,0.003030


In [183]:
df_cat.What_matters_most_to_you_in_choosing_a_course

0       0.999675
1       0.999675
2       0.999675
3       0.999675
4       0.999675
          ...   
9235    0.999675
9236    0.999675
9237    0.999675
9238    0.999675
9239    0.999675
Name: What_matters_most_to_you_in_choosing_a_course, Length: 9240, dtype: float64

In [184]:
df_cat.drop(labels=['Search','Magazine','Newspaper','Digital_Advertisement','Newspaper_Article','X_Education_Forums','Through_Recommendations','Lead_Quality',
           'Asymmetrique_Profile_Index','Asymmetrique_Activity_Index'],axis=1,inplace=True)

In [185]:
df_cat.shape


(9240, 22)

In [186]:
cleaned_advertising_dataset = numerical_cols_df.join(df_cat)

In [187]:
cleaned_advertising_dataset.to_csv('Cleaned_dataset.csv',index=False)